In [37]:
from skimage import data
from skimage.feature import Cascade
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np

Tutaj należy wybrać ustawienia funkcji

In [38]:
#ścieżka filmu
path_vid='C:/Users/ADMIN/Desktop/WIN_20221201_12_03_37_Pro.mp4'
#ustawienie szerokości kontura
szer=12

Dodawanie potrzebnych folderów i ścieżek 

In [39]:
#ścieżka do foldera na pulpicie
path0='C:/Users/ADMIN/Desktop/wykrywanie twarzy z filmu'   
#ścieżka do foldera z klatkami z obrazu
path1='C:/Users/ADMIN/Desktop/wykrywanie twarzy z filmu/zapisane obrazy'   
#ścieżka do foldera z wyciętymi twarzami
path2='C:/Users/ADMIN/Desktop/wykrywanie twarzy z filmu/twarz'


if not os.path.exists(path0):
    os.mkdir(path0)

if not os.path.exists(path2):
    os.mkdir(path2)

if not os.path.exists(path1):
    os.mkdir(path1)
    

In [40]:
capture = cv2.VideoCapture(path_vid)
 
frameNr = 0
 
while (True):
 
    success, frame = capture.read()
    if success:
        cv2.imwrite(path1+'/frame_{}.jpg'.format(frameNr),frame)
 
    else:
        break
 
    frameNr = frameNr+1
 
capture.release()
a=os.listdir(path1)
sort=[]
for elem in range(len(a)):
    min=np.inf
    for elem in a:
        elem=int(elem[elem.index('_')+1:elem.index('.')])
        if elem<min:
            min=elem
    elem='frame_'+str(min)+'.jpg'
    sort.append(elem)
    a.remove(elem)


Funkcja która otrzymuje ścieżkę do filmu, zwraca film zkładający się z wykrytych twarzy lub zastępczej planszy w przypadku nie wykrycia niczego 

In [41]:
none_image='C:/Users/ADMIN/Desktop/804-8044155_no-video-filled-icon.jpg'
def video2face(path):
    #cap=cv2.VideoCapture('C:/Users/ADMIN/Desktop/70nrrRD6iTddea9iciacLvcn1jBMQIIs.mp4')
    capture = cv2.VideoCapture(path)

    frameNr = 0

    while (True):

        success, frame = capture.read()
        if success:
            cv2.imwrite(path1+'/frame_{}.jpg'.format(frameNr),frame)

        else:
            break

        frameNr = frameNr+1
    capture.release()
    frameNr = 0
    detected_img=[]
    
    for elem in sort:
        trained_file=data.lbp_frontal_face_cascade_filename()
        detector=Cascade(trained_file)
        image=cv2.imread(path1+'/{}'.format(elem),cv2.COLOR_BGR2RGB)
        detected=detector.detect_multi_scale(img=image,scale_factor=1.1,step_ratio=1,min_size=(140,140),max_size=(500,500))
        detected_img.append(detected)
        if detected!=[]:
            cv2.imwrite(path2+'/face_{}.jpg'.format(frameNr),image[detected[0]['r']:detected[0]['r']+detected[0]['width'],detected[0]['c']:detected[0]['c']+detected[0]['height']])
        if detected==[]:
            cv2.imwrite(path2+'/face_{}.jpg'.format(frameNr),plt.imread(none_image))
        frameNr = frameNr+1
    return detected_img

In [42]:
detected_img=video2face(path_vid)

In [43]:
#tą funkcję trzebe zmodyfikować tak żeby działąła dla foldera 'twarz' i 'zapisane zdjęcia' tak samo,
#sposób: niech dalej robi to co robi byłko niech zapisuje wartości w sorciej jako sładowa
#'litery do '_'+ posortowana liczba + '.'


a=os.listdir(path1)
b=os.listdir(path2)
def sort_l(lista):
    sort=[]

    for elem in range(len(lista)):
        min=np.inf
        for elem in lista:
            elem1=int(elem[elem.index('_')+1:elem.index('.')])
            if elem1<min:
                min=elem1
        if elem.index('_')==5:
            elem1='frame_'+str(min)+'.jpg'
            sort.append(elem1)
            lista.remove(elem1)
        if elem.index('_')==4:
            elem1='face_'+str(min)+'.jpg'
            sort.append(elem1)
            lista.remove(elem1)
    return sort

In [44]:
img_list=sort_l(a)
face_list=sort_l(b)

In [45]:
a = cv2.imread(path1+'/frame_13.jpg', cv2.IMREAD_UNCHANGED)
b = cv2.imread(path2+'/face_2.jpg', cv2.IMREAD_UNCHANGED)

def scale(img1,img2):
    scale=img1.shape[0]/img2.shape[0]
    width = int(img2.shape[0] * scale )
    height = int(img2.shape[0] * scale )
    dim = (width, height)
  
    # resize image
    resized = cv2.resize(img2, dim, interpolation = cv2.INTER_AREA)
    return resized

In [46]:
for elem in range(len(img_list)):
    img_list[elem]=cv2.imread(path1+'/{}'.format(img_list[elem]))
for elem in range(len(face_list)):
    face_list[elem]=cv2.imread(path2+'/{}'.format(face_list[elem]))

In [47]:
for elem in range(len(img_list)):
    detected=detected_img[elem]
    if detected!=[]:
        img_list[elem][detected[0]['r']:detected[0]['r']+szer,detected[0]['c']:detected[0]['c']+detected[0]['width']]=1
        img_list[elem][detected[0]['r']+detected[0]['width']-szer:detected[0]['r']+detected[0]['width'],detected[0]['c']:detected[0]['c']+detected[0]['width']]=1
        img_list[elem][detected[0]['r']:detected[0]['r']+detected[0]['width'],detected[0]['c']:detected[0]['c']+szer]=1
        img_list[elem][detected[0]['r']:detected[0]['r']+detected[0]['width'],detected[0]['c']+detected[0]['width']-szer:detected[0]['c']+detected[0]['width']]=11


In [48]:
#to trzeba pare razy włączyć z jakiegoś powodu, można tu dopisać pętle
for elem in range(6):
    for elem in range(len(img_list)):
        face_list[elem]=scale(img_list[elem],face_list[elem])
for elem in range(len(img_list)):
    face_list[elem]=np.concatenate((img_list[elem], face_list[elem]), axis=1)


In [49]:
for elem in range(len(face_list)):
    cv2.imwrite(path2+'/face_{}.jpg'.format(elem),face_list[elem])

In [50]:
import cv2
import os

image_folder = path2
video_name = path0+'/video_1.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 30, (width,height))
images=sort_l(images)
for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()